In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import requests
import torch
import re
from bs4 import BeautifulSoup


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [ ]:
tokens = tokenizer.encode("I hate you", return_tensors = 'pt')
tokens
token_decode = tokenizer.decode(tokens[0])
token_decode

'[CLS] i hate you [SEP]'

In [ ]:
result = model(tokens)
ratings = int(torch.argmax(result.logits)+1)
ratings


1

In [ ]:
request = requests.get("https://www.yelp.com/biz/babette-berkeley?hrid=WNBiHdrEM2VlvCiRoK11JQ")

soup = BeautifulSoup(request.text,'html.parser')

class_find = re.compile(".*comment.*")
results = soup.find_all('p', {'class':class_find})
reviews = [result.text for result in results]



In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.array(reviews), columns = ["reviews"]) # dont do columns = "reviews", must pass a list here
df.head()

,reviews
0,"Fabulous find in Berkeley, enjoyed one of the ..."
1,What a lovely place!Brunch was delicious! The ...
2,I will be back!Ambiance: what a cute patio. Th...
3,We stumbled across Babette after the wait at L...
4,Excellent food! Especially love the pancakes :...


In [ ]:
def find_rating(reviews):

  tokens = tokenizer.encode(reviews, return_tensors = 'pt')
  result = model(tokens)
  ratings = int(torch.argmax(result.logits+1)) #logits defines the ratings of the sentence, here we do argmax, to show the rating with the highest probable value.
  return ratings

rating = find_rating(df["reviews"].iloc[1])
rating


4

0    Fabulous find in Berkeley, enjoyed one of the ...
1    What a lovely place!Brunch was delicious! The ...
2    I will be back!Ambiance: what a cute patio. Th...
3    We stumbled across Babette after the wait at L...
4    Excellent food! Especially love the pancakes :...
5    We went here for brunch. Good, unique food! Se...
6    Had an amazing brunch experience here today! A...
7    I hosted my friends 70th birthday party on Jan...
8    good drinks-good music playlist inside-heaters...
9    One of the most unique brunch spots I've been ...
Name: reviews, dtype: object

In [ ]:

df["ratings"] = df["reviews"].apply(lambda x : find_rating(x[:512])) #limting to 512 ratings right now



df.head()

,reviews,ratings
0,"Fabulous find in Berkeley, enjoyed one of the ...",4
1,What a lovely place!Brunch was delicious! The ...,4
2,I will be back!Ambiance: what a cute patio. Th...,2
3,We stumbled across Babette after the wait at L...,4
4,Excellent food! Especially love the pancakes :...,4


In [ ]:
average_rating = df["ratings"].mean()

average_rating

3.6